In [4]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Fungsi keanggotaan linear turun dan naik yang diperbaiki
def turun_scalar(a, b, x):
    return np.where(x <= b, 1, 
                    np.where(x >= a, 0, 
                             (a - x) / (a - b)))

def naik_scalar(a, b, x):
    return np.where(x <= a, 0, 
                    np.where(x >= b, 1, 
                             (x - a) / (b - a)))

# Agregasi (inversi fungsi naik/turun)
def agregasi_turun(a, b, alfa):
    return a - (alfa * (a - b))

def agregasi_naik(a, b, alfa):
    return alfa * (b - a) + a

# Input user
suhu_luar = float(input("Masukkan Suhu Luar (10-40 °C): "))
suhu_dalam = float(input("Masukkan Suhu Dalam (10-40 °C): "))
jumlah_orang = int(input("Masukkan Jumlah Orang (0-50): "))

# Validasi input
if not (10 <= suhu_luar <= 40):
    print("Suhu luar harus antara 10 dan 40 °C.")
    sys.exit()
if not (10 <= suhu_dalam <= 40):
    print("Suhu dalam harus antara 10 dan 40 °C.")
    sys.exit()
if not (0 <= jumlah_orang <= 50):
    print("Jumlah orang harus antara 0 dan 50.")
    sys.exit()


# Fuzzifikasi
suhu_luar_fuzzy = {
    'sejuk': turun_scalar(24, 10, suhu_luar),
    'normal': np.minimum(naik_scalar(22, 24, suhu_luar), turun_scalar(33, 22, suhu_luar)),
    'panas': naik_scalar(32, 40, suhu_luar)
}

suhu_dalam_fuzzy = {
    'dingin': turun_scalar(24, 10, suhu_dalam),
    'normal': np.minimum(naik_scalar(22, 24, suhu_dalam), turun_scalar(31, 22, suhu_dalam)),
    'hangat': naik_scalar(30, 40, suhu_dalam)
}

jumlah_orang_fuzzy = {
    'sedikit': turun_scalar(18, 0, jumlah_orang),
    'sedang': np.minimum(naik_scalar(12, 34, jumlah_orang), turun_scalar(34, 12, jumlah_orang)),
    'banyak': naik_scalar(30, 50, jumlah_orang)
}

# Output kategori suhu optimal dengan rentang yang diperbarui
kategori_output = {
    'dingin': (17, 15),
    'cukup_dingin': (19, 17),
    'normal': (21, 19),
    'cukup_hangat': (23, 21),
    'hangat': (25, 23)
}

# Aturan fuzzy
aturan = [
    ('sejuk', 'dingin', 'sedikit', 'cukup_dingin'),
    ('sejuk', 'dingin', 'sedang', 'normal'),
    ('sejuk', 'dingin', 'banyak', 'cukup_hangat'),
    ('sejuk', 'normal', 'sedikit', 'normal'),
    ('sejuk', 'normal', 'sedang', 'cukup_hangat'),
    ('sejuk', 'normal', 'banyak', 'hangat'),
    ('sejuk', 'hangat', 'sedikit', 'cukup_hangat'),
    ('sejuk', 'hangat', 'sedang', 'hangat'),
    ('sejuk', 'hangat', 'banyak', 'hangat'),
    ('normal', 'dingin', 'sedikit', 'normal'),
    ('normal', 'dingin', 'sedang', 'cukup_hangat'),
    ('normal', 'dingin', 'banyak', 'hangat'),
    ('normal', 'normal', 'sedikit', 'cukup_hangat'),
    ('normal', 'normal', 'sedang', 'hangat'),
    ('normal', 'normal', 'banyak', 'hangat'),
    ('normal', 'hangat', 'sedikit', 'hangat'),
    ('normal', 'hangat', 'sedang', 'hangat'),
    ('normal', 'hangat', 'banyak', 'hangat'),
    ('panas', 'dingin', 'sedikit', 'cukup_dingin'),
    ('panas', 'dingin', 'sedang', 'dingin'),
    ('panas', 'dingin', 'banyak', 'dingin'),
    ('panas', 'normal', 'sedikit', 'cukup_dingin'),
    ('panas', 'normal', 'sedang', 'dingin'),
    ('panas', 'normal', 'banyak', 'dingin'),
    ('panas', 'hangat', 'sedikit', 'dingin'),
    ('panas', 'hangat', 'sedang', 'dingin'),
    ('panas', 'hangat', 'banyak', 'dingin'),
]

# Evaluasi aturan
z_list = []
alfa_list = []

for rule in aturan:
    sl, sd, jo, so = rule
    alfa = min(suhu_luar_fuzzy[sl], suhu_dalam_fuzzy[sd], jumlah_orang_fuzzy[jo])
    alfa_list.append(alfa)
    atas, bawah = kategori_output[so]
    
    # Jika output adalah 'dingin' atau 'cukup_dingin', maka agregasi turun
    if so in ['dingin', 'cukup_dingin']:
        z = agregasi_turun(atas, bawah, alfa)
    else:
        z = agregasi_naik(bawah, atas, alfa)
    
    z_list.append(z)

# Defuzzifikasi
if sum(alfa_list) != 0:
    suhu_optimal = sum(a*z for a, z in zip(alfa_list, z_list)) / sum(alfa_list)
else:
    suhu_optimal = 0
    
# Menampilkan informasi inputan dari user
print("\n Input Pengguna:")
print(f"- Suhu Luar       : {suhu_luar}°C")
print(f"- Suhu Dalam      : {suhu_dalam}°C")
print(f"- Jumlah Orang    : {jumlah_orang} orang")

print("\nSuhu Optimal AC yang disarankan adalah: {:.2f}°C".format(suhu_optimal))

# Visualisasi Fungsi Keanggotaan
# Suhu Luar
x = np.linspace(10, 40, 500)
plt.figure()
plt.plot(x, turun_scalar(24, 10, x), label='Sejuk')
plt.plot(x, np.minimum(naik_scalar(22, 24, x), turun_scalar(33, 22, x)), label='Normal')
plt.plot(x, naik_scalar(32, 40, x), label='Panas')
plt.title('Fungsi Keanggotaan - Suhu Luar')
plt.xlabel('Suhu (°C)')
plt.ylabel('Derajat Keanggotaan')
plt.grid(True)
plt.legend()
plt.show()

# Suhu Dalam
x = np.linspace(10, 40, 500)
plt.figure()
plt.plot(x, turun_scalar(24, 10, x), label='Dingin')
plt.plot(x, np.minimum(naik_scalar(22, 24, x), turun_scalar(31, 22, x)), label='Normal')
plt.plot(x, naik_scalar(30, 40, x), label='Hangat')
plt.title('Fungsi Keanggotaan - Suhu Dalam')
plt.xlabel('Suhu (°C)')
plt.ylabel('Derajat Keanggotaan')
plt.grid(True)
plt.legend()
plt.show()

# Jumlah Orang
x = np.linspace(0, 50, 500)
plt.figure()
plt.plot(x, turun_scalar(18, 0, x), label='Sedikit')
plt.plot(x, np.minimum(naik_scalar(12, 34, x), turun_scalar(34, 12, x)), label='Sedang')
plt.plot(x, naik_scalar(30, 50, x), label='Banyak')
plt.title('Fungsi Keanggotaan - Jumlah Orang')
plt.xlabel('Jumlah Orang')
plt.ylabel('Derajat Keanggotaan')
plt.grid(True)
plt.legend()
plt.show()

# Output: Suhu Optimal
x = np.linspace(15, 25, 500)
plt.figure()
plt.plot(x, turun_scalar(17, 15, x), label='Dingin')
plt.plot(x, turun_scalar(19, 17, x), label='Cukup Dingin')
plt.plot(x, np.minimum(naik_scalar(19, 21, x), turun_scalar(21, 19, x)), label='Normal')
plt.plot(x, naik_scalar(21, 23, x), label='Cukup Hangat')
plt.plot(x, naik_scalar(23, 25, x), label='Hangat')

# Menandai suhu optimal pada grafik
plt.axvline(suhu_optimal, color='red', linestyle='--', label=f'Suhu Optimal: {suhu_optimal:.2f}°C')

plt.title('Fungsi Keanggotaan - Suhu Optimal')
plt.xlabel('Suhu (°C)')
plt.ylabel('Derajat Keanggotaan')
plt.grid(True)
plt.legend()
plt.show()

Suhu luar harus antara 10 dan 40 °C.


SystemExit: 